In [1]:
import os
import scipy.io as sio 
import path
import pandas as pd
import numpy as np
import collections
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pyspark

In [3]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("msg10").getOrCreate()

In [2]:
%matplotlib inline

In [3]:
import tensorflow as tf
tf.__version__

D:\environment\python\anaconda352\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.12.0'

数据说明：
1. 对于PSG数据
   - stage 睡眠所处阶段
   - 3 -> 醒来， 2 -> 浅睡， 1 -> 深睡

In [4]:
data_root=r"E:\datasources\MS10Data"
p = r"E:\datasources\MS10Data\1\psg\psg.mat"
rdp=r"E:\datasources\MS10Data\1\raw\rawMat.mat"
save_data=r"E:\datasources\MS10DataTF"

In [5]:
psg_data_paths=[os.path.join(data_root,f"{i}\\psg\\psg.mat") for i in range(1,33)]
raw_data_paths=[os.path.join(data_root,f"{i}\\raw\\rawMat.mat") for i in range(1,33)]
psg_raw_data_paths=list(zip(psg_data_paths,raw_data_paths))

In [6]:
def parse_mat_psg(fp):
    psg1=sio.loadmat(fp)
    ti=np.apply_along_axis(lambda row:row[0],0,psg1["psgStage"]['Time']).flatten()
    st=np.apply_along_axis(lambda row:row[0],0,psg1["psgStage"]['stage']).flatten()
    st=np.apply_along_axis(lambda x:int(x),1,st.reshape(-1,1))
    print(st.shape)
    return pd.DataFrame(np.vstack([ti,st]).transpose(),columns=['time','stage'])

def parse_mat_signal(fp):
    rs=sio.loadmat(fp)
    r1=rs['rawData']['dataSignalOld'][0,0]
    r2=rs['rawData']['dataSignal'][0,0]
    r3=rs['rawData']['dataSwitch'][0,0]
    r4=rs['rawData']['Time'][0,0]
    return pd.DataFrame(np.hstack([r1,r2,r3,r4]),columns=['data_signal_old','data_signal','data_switch','time'])

def parse_mat_time(matlab_datenum):
    return datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1) - timedelta(days = 366)
    

In [7]:
# 数据解析生成tf-record
# 定义特征对象
def _int64_feature(value):
    """生成一个对应类型的特征对象"""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    """生成一个对应类型的特征对象"""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """生成一个对应类型的特征对象"""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _make_named_features(named_features):
    """生成一个命名的特征，给特征对象赋予一个schema"""
    return tf.train.Features(feature=named_features)

# 定义序列record，多了一层包裹的抽象
def _feature_list(feature_list):
    """生成一个特征对象的序列"""
    return tf.train.FeatureList(feature=feature_list)

def _make_named_featurelists(named_featurelists):
    """生成一个命名的特征，给特征对象的序列赋予一个schema"""
    return tf.train.FeatureLists(feature_list=named_featurelists)

In [8]:
plt.figure(figsize=(100,50))
plt.plot(signal['data_signal'],)
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)


NameError: name 'signal' is not defined

<Figure size 7200x3600 with 0 Axes>

In [9]:
signal=parse_mat_signal(rdp)
psg=parse_mat_psg(p)

psg_time=psg['time'].values
psg_stage=psg['stage'].values
psg_int=list(zip(psg_time[:-1],psg_time[1:],psg_stage[1:]))

def make_signal_psg_list(psg,signal):
    signal_list=[]
    psg_list=[]
    psg_time=psg['time'].values
    psg_stage=psg['stage'].values
    psg_int=list(zip(psg_time[:-1],psg_time[1:],psg_stage[1:]))
    for it in psg_int:
        signal_list.append(signal.query(f"(time>{it[0]})and(time<{it[1]})")['data_signal'].values)
        psg_list.append(it[2])
    return signal_list,psg_list

(1116,)


In [10]:
signal_list,psg_list=make_signal_psg_list(psg,signal)

In [3]:
# 从字典列表创建df

l = [('Alice', 1)]
spark.createDataFrame(l).show()
spark.createDataFrame(l, ['name', 'age']).show()
d = [{'name': 'Alice', 'age': 1}]
rs=spark.createDataFrame(d).show()

+-----+---+
|   _1| _2|
+-----+---+
|Alice|  1|
+-----+---+

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



D:\environment\python\anaconda352\lib\site-packages\pyspark\sql\session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+---+-----+
|age| name|
+---+-----+
|  1|Alice|
+---+-----+



In [13]:
from pyspark.sql import Row

# 从rdd创建df
rdd = spark.sparkContext.parallelize(l)
spark.createDataFrame(rdd).collect()
df = spark.createDataFrame(rdd, ['name', 'age'])
df.collect()

Person = Row('name', 'age')
person = rdd.map(lambda r: Person(*r))
df2 = spark.createDataFrame(person)
df2.collect()

[Row(name='Alice', age=1)]

In [14]:
from pyspark.sql.types import *
# 指明schema 结合rdd创建df
schema = StructType([StructField("name", StringType(), True),StructField("age", IntegerType(), True)])
df3 = spark.createDataFrame(rdd, schema)
df3.collect()

[Row(name='Alice', age=1)]

In [40]:
# 从 pandas 创建
spark.createDataFrame(df.toPandas()).show()
spark.createDataFrame(rdd, "a: string, b: int").show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+

+-----+---+
|    a|  b|
+-----+---+
|Alice|  1|
+-----+---+



In [37]:
df.toPandas()

,name,age
0,Alice,1


In [50]:
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice|  1|
+-----+---+



In [54]:
df.createOrReplaceTempView("table1")
df2 = spark.sql("SELECT name AS f1, age as f2 from table1")
df2.collect()

[Row(f1='Alice', f2=1)]

In [56]:
df.createOrReplaceTempView("table1")
df2 = spark.table("table1")
sorted(df.collect()) == sorted(df2.collect())

True

In [60]:
strlen = spark.udf.register("stringLengthString", lambda x: len(x))
spark.sql("SELECT stringLengthString('test')").show()

+------------------------+
|stringLengthString(test)|
+------------------------+
|                       4|
+------------------------+



In [59]:
spark.sql("SELECT 'foo' AS text").select(strlen("text")).show()
df.select(strlen("name")).show()

<function __main__.<lambda>(x)>

In [77]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
import random

_ = spark.udf.register("stringLengthInt", lambda x: x*3)
spark.sql("SELECT stringLengthInt('test')").show()

slen = udf(lambda s: len(s), IntegerType())
_= spark.udf.register("slen", slen)
spark.sql("SELECT slen('test')").show()

+---------------------+
|stringLengthInt(test)|
+---------------------+
|         testtesttest|
+---------------------+

+----------+
|slen(test)|
+----------+
|         4|
+----------+



In [80]:
random_udf = udf(lambda: random.randint(0, 100), IntegerType()).asNondeterministic()
new_random_udf = spark.udf.register("random_udf", random_udf)

In [84]:
spark.sql("SELECT 'random_udf' as text").select(new_random_udf()).show()

+----------+
|<lambda>()|
+----------+
|        39|
+----------+



In [ ]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
@pandas_udf("integer", PandasUDFType.SCALAR)  # doctest: +SKIP
def add_one(x):
    return x + 1
_ = spark.udf.register("add_one", add_one)  # doctest: +SKIP
spark.sql("SELECT add_one(id) FROM range(3)").show()  # doctest: +SKIP

# @pandas_udf("integer", PandasUDFType.GROUPED_AGG)  # doctest: +SKIP
# def sum_udf(v):
#     return v.sum()

# _ = spark.udf.register("sum_udf", sum_udf)  # doctest: +SKIP
# q = "SELECT sum_udf(v1) FROM VALUES (3, 0), (2, 0), (1, 1) tbl(v1, v2) GROUP BY v2"
# spark.sql(q).show()  # doctest: +SKIP

In [ ]:
from pyspark.sql.types import IntegerType
spark.udf.registerJavaFunction("javaStringLength", "test.org.apache.spark.sql.JavaStringLength", IntegerType())
spark.sql("SELECT javaStringLength('test')").show()

In [13]:
df.agg({"age": "max"}).show()
from pyspark.sql import functions as F
df.agg(F.min(df.age)).show()

+--------+
|max(age)|
+--------+
|       1|
+--------+

+--------+
|min(age)|
+--------+
|       1|
+--------+



In [9]:
from pyspark.sql.functions import *
df_as1 = df.alias("df_as1")
df_as2 = df.alias("df_as2")
joined_df = df_as1.join(df_as2, col("df_as1.name") == col("df_as2.name"), 'inner')
joined_df.select("df_as1.name", "df_as2.name", "df_as2.age").collect()

NameError: name 'df' is not defined

In [39]:
df1 = spark.createDataFrame([("a", 11), ("a", 5), ("a", 7), ("a", 2), ("b",  3), ("c", 4)], ["C1", "C2"])
df2 = spark.createDataFrame([("a", 1), ("b", 3)], ["C1", "C2"])

In [9]:
df1.filter("C2>1").show()
df1.where("C2>1").show()
df1.select(col("C2")>1).show()

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a|  5|
|  a|  7|
|  a|  2|
|  b|  3|
|  c|  4|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a|  5|
|  a|  7|
|  a|  2|
|  b|  3|
|  c|  4|
+---+---+



NameError: name 'col' is not defined

In [20]:
df1.sort(df1["C2"].desc()).show()
df1.sort("C2", ascending=False).show()
df1.orderBy(df1["C2"].desc()).show()
from pyspark.sql.functions import *
df1.sort(asc("C2")).show()
df1.orderBy(desc("C2"), "C1").show()
df1.orderBy(["C2", "C1"], ascending=[0, 1]).show()

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a|  7|
|  a|  5|
|  c|  4|
|  b|  3|
|  a|  2|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a|  7|
|  a|  5|
|  c|  4|
|  b|  3|
|  a|  2|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a|  7|
|  a|  5|
|  c|  4|
|  b|  3|
|  a|  2|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a|  2|
|  b|  3|
|  c|  4|
|  a|  5|
|  a|  7|
|  a| 11|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a|  7|
|  a|  5|
|  c|  4|
|  b|  3|
|  a|  2|
+---+---+

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a|  7|
|  a|  5|
|  c|  4|
|  b|  3|
|  a|  2|
+---+---+



In [29]:
df1.na.replace(5,20,'C2').show()

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a| 20|
|  a|  7|
|  a|  2|
|  b|  3|
|  c|  4|
+---+---+



In [31]:
dataset = spark.range(0, 100).select((col("id") % 3).alias("key"))
sampled = dataset.sampleBy("key", fractions={0: 0.1, 1: 0.2}, seed=0)
sampled.groupBy("key").count().orderBy("key").show()

+---+-----+
|key|count|
+---+-----+
|  0|    3|
|  1|    7|
+---+-----+



In [39]:
gdf = df1.groupBy(df1.C1)
gdf.agg({"*": "count"}).show()

+---+--------+
| C1|count(1)|
+---+--------+
|  c|       1|
|  b|       1|
|  a|       4|
+---+--------+



In [5]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
df = spark.createDataFrame([(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)],("id", "v"))
@pandas_udf("id long, v double", PandasUDFType.GROUPED_MAP)  # doctest: +SKIP
def normalize(pdf):
    v = pdf.v
    return pdf.assign(v=(v - v.mean()) / v.std())
df.groupby("id").apply(normalize).show()  # doctest: +SKIP

+---+-------------------+
| id|                  v|
+---+-------------------+
|  1|-0.7071067811865475|
|  1| 0.7071067811865475|
|  2|-0.8320502943378437|
|  2|-0.2773500981126146|
|  2| 1.1094003924504583|
+---+-------------------+



In [6]:
df = spark.createDataFrame([([1, 2], {"key": "value"})], ["l", "d"])
df.show()
df.select(df.l.getItem(0), df.d.getItem("key")).show()

+------+--------------+
|     l|             d|
+------+--------------+
|[1, 2]|[key -> value]|
+------+--------------+

+----+------+
|l[0]|d[key]|
+----+------+
|   1| value|
+----+------+



In [72]:
from pyspark.sql import Window
from pyspark.sql import functions as F
window = Window.partitionBy("C1").orderBy("C2")
df1.show()
window2 = Window.partitionBy("C1").orderBy("C2").rowsBetween(-1,1)
df1.select(F.dense_rank().over(window).alias("c1_")).show()
df1.select(F.min("C2").over(window2).alias("c2_")).show()

+---+---+
| C1| C2|
+---+---+
|  a| 11|
|  a|  5|
|  a|  7|
|  a|  2|
|  b|  3|
|  c|  4|
+---+---+

+---+
|c1_|
+---+
|  1|
|  1|
|  1|
|  2|
|  3|
|  4|
+---+

+---+
|c2_|
+---+
|  4|
|  3|
|  2|
|  2|
|  5|
|  7|
+---+



In [44]:
window.rowsBetween(-1,1)

In [13]:
p=r"E:\GOOD_PROJECTS\spark-master\python\test_support\sql\ages.csv"

In [16]:
orders = spark.createDataFrame([("o1", "s1", "2017-05-01", 100), ("o2", "s1", "2017-05-02", 200), ("o3", "s2", "2017-05-01", 300)],
                              ["order_id", "seller_id", "order_date", "price"])

In [17]:
orders.show()

+--------+---------+----------+-----+
|order_id|seller_id|order_date|price|
+--------+---------+----------+-----+
|      o1|       s1|2017-05-01|  100|
|      o2|       s1|2017-05-02|  200|
|      o3|       s2|2017-05-01|  300|
+--------+---------+----------+-----+



In [57]:
rankSpec = Window.partitionBy("seller_id").orderBy("order_date")

shopOrderRank = orders.withColumn("rank", F.dense_rank().over(rankSpec)).show()  

sumSpec = Window.partitionBy("seller_id").orderBy("order_date").rowsBetween(-1, 0)

orders.withColumn("cumulative_sum", F.sum("price").over(sumSpec)).show()

+--------+---------+----------+-----+----+
|order_id|seller_id|order_date|price|rank|
+--------+---------+----------+-----+----+
|      o3|       s2|2017-05-01|  300|   1|
|      o1|       s1|2017-05-01|  100|   1|
|      o2|       s1|2017-05-02|  200|   2|
+--------+---------+----------+-----+----+

+--------+---------+----------+-----+--------------+
|order_id|seller_id|order_date|price|cumulative_sum|
+--------+---------+----------+-----+--------------+
|      o3|       s2|2017-05-01|  300|           300|
|      o1|       s1|2017-05-01|  100|           100|
|      o2|       s1|2017-05-02|  200|           300|
+--------+---------+----------+-----+--------------+



In [97]:
data=[("站点1", "2017-01-01", 50),("站点1", "2017-01-02", 45), ("站点1", "2017-01-03", 55), 
 ("站点2", "2017-01-01", 25), ("站点2", "2017-01-02", 29),("站点2", "2017-01-03", 27)]

schema=("site", "date", "user_cnt")
df_w=spark.createDataFrame(data,schema)

#wSpec = window.partitionBy("site").orderBy("date").rowsBetween(-1, 1)
wSpec = window.partitionBy().orderBy("date").rowsBetween(-1, 1)

In [98]:
df_w.show()
df_w.withColumn("movingAvg",F.avg(df_w["user_cnt"]).over(wSpec)).show()

+----+----------+--------+
|site|      date|user_cnt|
+----+----------+--------+
| 站点1|2017-01-01|      50|
| 站点1|2017-01-02|      45|
| 站点1|2017-01-03|      55|
| 站点2|2017-01-01|      25|
| 站点2|2017-01-02|      29|
| 站点2|2017-01-03|      27|
+----+----------+--------+

+----+----------+--------+---------+
|site|      date|user_cnt|movingAvg|
+----+----------+--------+---------+
| 站点1|2017-01-01|      50|     37.5|
| 站点2|2017-01-01|      25|     40.0|
| 站点1|2017-01-02|      45|     33.0|
| 站点2|2017-01-02|      29|     43.0|
| 站点1|2017-01-03|      55|     37.0|
| 站点2|2017-01-03|      27|     41.0|
+----+----------+--------+---------+



In [87]:
rdd_w=spark.sparkContext.parallelize([50,45,55])
rdd_w.map(lambda x:x+10).collect()

[60, 55, 65]

In [102]:
from pyspark.sql.functions import arrays_zip
df0 = spark.createDataFrame([(([1, 2, 3], [2, 3, 4]))], ['vals1', 'vals2'])
df0.select(arrays_zip(df0.vals1, df.vals2).alias('zipped')).collect()

ImportError: cannot import name 'arrays_zip'

In [103]:
df = spark.createDataFrame([(1, ["foo", "bar"], {"x": 1.0}), (2, [], {}), (3, None, None)],("id", "an_array", "a_map"))
df.show()
df.select("id", "an_array", explode_outer("a_map")).show()

+---+----------+----------+
| id|  an_array|     a_map|
+---+----------+----------+
|  1|[foo, bar]|[x -> 1.0]|
|  2|        []|        []|
|  3|      null|      null|
+---+----------+----------+

+---+----------+----+-----+
| id|  an_array| key|value|
+---+----------+----+-----+
|  1|[foo, bar]|   x|  1.0|
|  2|        []|null| null|
|  3|      null|null| null|
+---+----------+----+-----+



In [105]:
from pyspark.sql.functions import map_keys
df = spark.sql("SELECT map(1, 'a', 2, 'b') as data")
df.show()
df.select(map_keys("data").alias("keys")).show()

+----------------+
|            data|
+----------------+
|[1 -> a, 2 -> b]|
+----------------+

+------+
|  keys|
+------+
|[1, 2]|
+------+



In [106]:
df.collect()

[Row(data={1: 'a', 2: 'b'})]

In [107]:
df.take(1)

[Row(data={1: 'a', 2: 'b'})]

In [139]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType, StringType
slen = pandas_udf(lambda s: s.str.len(), IntegerType())  # doctest: +SKIP
@pandas_udf(StringType())  # doctest: +SKIP
def to_upper(s):
    # 输入和输出是等长是series
    #print(s1.values,s.__class__)
    return str(type(s))[-8:]+s.str.upper()
@pandas_udf("integer", PandasUDFType.SCALAR)  # doctest: +SKIP
def add_one(x):
    print(x.values,x.__class__)
    return x + 1

df = spark.createDataFrame([(1, "John Doe", 21)],("id", "name", "age"))  # doctest: +SKIP
df.select(slen("name").alias("slen(name)"), to_upper("name"), add_one("age")).show()  # doctest: +SKIP

+----------+----------------+------------+
|slen(name)|  to_upper(name)|add_one(age)|
+----------+----------------+------------+
|         8|Series'>JOHN DOE|          22|
+----------+----------------+------------+



In [128]:
import pandas as pd

In [145]:
df = spark.createDataFrame([(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)],("id", "v"))  # doctest: +SKIP
@pandas_udf("id long, v double", PandasUDFType.GROUPED_MAP)  # doctest: +SKIP
def normalize(pdf):
    v = pdf.v
    return pdf.assign(v=(v - v.mean()) / v.std())
df.groupby("id").apply(normalize).show() 

+---+-------------------+
| id|                  v|
+---+-------------------+
|  1|-0.7071067811865475|
|  1| 0.7071067811865475|
|  2|-0.8320502943378437|
|  2|-0.2773500981126146|
|  2| 1.1094003924504583|
+---+-------------------+



In [136]:
s1.values

array(['a33', 'b33', 'c33', 'd33', 'e33'], dtype=object)

In [146]:
pdf= pd.DataFrame([(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)],columns=("id", "v"))
pd.DataFrame(["key" + (pdf.v.mean(),)])

TypeError: must be str, not tuple

In [148]:
f1 = spark.createDataFrame([(-2, 2)], ('C1', 'C2'))
df1.select(sequence('C1', 'C2').alias('r')).collect()
df2 = spark.createDataFrame([(4, -4, -2)], ('C1', 'C2', 'C3'))
df2.show()
df2.select(F.sequence('C1', 'C2', 'C3').alias('r')).show()

NameError: name 'sequence' is not defined

In [161]:
myf=udf(len)
strlen = spark.udf.register("myf", lambda x: len(x))
spark.sql("Select myf('abc')")

DataFrame[myf(abc): string]

In [ ]:
def multiply_func(a, b):
    return a * b

multiply = F.pandas_udf(multiply_func, returnType=sp_types.FloatType())

x = pd.Series([1, 2, 3])
print(multiply_func(x, x))
tmp_df=pd.DataFrame(np.arange(12).reshape(-1,3),columns=list('abc'))
tmp_df=tmp_df.assign(gp=[1,1,2,2])
tmp_sdf=spark.createDataFrame(tmp_df)
tmp_sdf.show()
tmp_sdfg=tmp_sdf.groupBy("gp")

# 使用框架内置的聚合函数实现简单的用户自定义的聚合函数
def my_af(col_name):
    coln=F.max(F.col(col_name))+F.max(F.col(col_name))+10
    return coln.alias("c2")

tmp_sdfg.agg(F.max(F.col("c"))*F.min(F.col("c"))).show()
tmp_sdfg.agg(my_af('c')).show()



from pyspark.sql.functions import pandas_udf, PandasUDFType
df = spark.createDataFrame([(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)],("id", "v"))  # doctest: +SKIP
#@pandas_udf("id long, v double, v1 double, v2 double", PandasUDFType.GROUPED_MAP)  # doctest: +SKIP
@pandas_udf("id long, v double", PandasUDFType.GROUPED_MAP)  # doctest: +SKIP
def normalize(pdf):
    v = pdf.v
    print(pdf)
    print(v.__class__,pdf.columns)
    #res=pdf.assign(v1=(v - v.mean()) / v.std(),v2=v.mean())
    res = pd.DataFrame(data=[pdf.id[0],v.mean()]).T
    print(res)
    return res
df.groupby("id").apply(normalize).show()


In [ ]:
tmp_df=pd.DataFrame(np.arange(12).reshape(-1,3),columns=list('abc'))
tmp_df=tmp_df.assign(gp=[1,1,2,2])
tmp_sdf=spark.createDataFrame(tmp_df)
tmp_sdf.show()
tmp_sdfg=tmp_sdf.groupBy("gp")
# 使用框架内置的聚合函数实现简单的用户自定义的聚合函数

mean_af=F.pandas_udf(lambda s:pd.Series([s.mean()]*s.size),returnType=sp_types.FloatType(),functionType=F.PandasUDFType.SCALAR)

def my_af(col_name):
    coln=F.max(F.col(col_name))+F.max(F.col(col_name))+10
    return coln.alias("c2")
tmp_sdfg.agg(F.max(F.col("c"))*F.min(F.col("c"))).show()
tmp_sdfg.agg(my_af('c')).show()
tmp_sdfg.agg(F.collect_list('c').alias('value_list')).show()
